In [ ]:
"""
Project Manager Agent Implementation
- Manages task assignment
- Monitors programmer progress
- Provides feedback and guidance
- Manages learning programs for programmers
"""

import os
import json
import asyncio
import logging
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime, timedelta
import uuid
from pydantic import BaseModel, Field
import numpy as np

# Import from base framework
from base_agent_framework import (
    BaseAgent, AgentRole, AgentStatus, Context, ContextType,
    Thought, ThoughtType, Memory
)

# LLM integration - using Phi-3-mini
try:
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
except ImportError:
    logging.error("Required packages not found. Please install with: pip install torch transformers")
    raise

# ====== Task and Project Models ======

class TaskStatus(Enum):
    """Possible statuses for a task"""
    PENDING = auto()
    ASSIGNED = auto()
    IN_PROGRESS = auto()
    REVIEW = auto()
    COMPLETED = auto()
    BLOCKED = auto()

class TaskPriority(Enum):
    """Priority levels for tasks"""
    LOW = auto()
    MEDIUM = auto()
    HIGH = auto()
    CRITICAL = auto()

class Task(BaseModel):
    """Represents a task in the system"""
    task_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    title: str
    description: str
    priority: TaskPriority
    status: TaskStatus = TaskStatus.PENDING
    created_at: datetime = Field(default_factory=datetime.now)
    due_date: Optional[datetime] = None
    assigned_to: Optional[str] = None
    estimated_effort: float = 1.0  # In hours
    actual_effort: float = 0.0
    dependencies: List[str] = Field(default_factory=list)  # List of task_ids
    required_expertise: List[str] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

class Project(BaseModel):
    """Represents a project in the system"""
    project_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    name: str
    description: str
    start_date: datetime = Field(default_factory=datetime.now)
    end_date: Optional[datetime] = None
    tasks: Dict[str, Task] = Field(default_factory=dict)
    team_members: List[str] = Field(default_factory=list)
    status: str = "PLANNING"
    metadata: Dict[str, Any] = Field(default_factory=dict)

# ====== Programmer Model ======

class ProgrammerProfile(BaseModel):
    """Profile of a programmer agent"""
    agent_id: str
    expertise: List[str] = Field(default_factory=list)
    skill_levels: Dict[str, float] = Field(default_factory=dict)  # Skill: level (0.0-1.0)
    current_tasks: List[str] = Field(default_factory=list)
    completed_tasks: List[str] = Field(default_factory=list)
    performance_metrics: Dict[str, float] = Field(default_factory=dict)
    learning_path: List[str] = Field(default_factory=list)
    availability: float = 1.0  # 0.0-1.0, percentage of time available

# ====== Project Manager Agent ======

class ProjectManagerAgent(BaseAgent):
    """Project Manager Agent implementation"""
    
    def __init__(self, agent_id: str, model_path: str):
        super().__init__(agent_id, AgentRole.PROJECT_MANAGER, model_path)
        self.projects: Dict[str, Project] = {}
        self.current_project_id: Optional[str] = None
        self.programmer_profiles: Dict[str, ProgrammerProfile] = {}
        self.specialists: Dict[str, str] = {}  # Specialist type: agent_id
        
        # Subscribe to relevant contexts
        self.subscribe_to_context(ContextType.MESSAGE)
        self.subscribe_to_context(ContextType.TASK)
        self.subscribe_to_context(ContextType.SYSTEM)
        self.subscribe_to_context(ContextType.LEARNING)
        
        # Prompt templates
        self.prompt_templates = {
            "task_assignment": """
You are a Project Manager agent responsible for assigning tasks to programmers.
Consider each programmer's expertise, current workload, and skill level.

Project: {project_name}
Task: {task_title} - {task_description}
Required expertise: {required_expertise}

Available programmers:
{programmer_profiles}

Based on this information, which programmer should be assigned to this task?
Provide your decision and brief reasoning.
""",
            "progress_evaluation": """
You are a Project Manager agent evaluating a programmer's progress.
Review the progress report and provide feedback.

Programmer: {programmer_id}
Task: {task_title} - {task_description}
Progress report: {progress_report}

Current status: {task_status}
Time elapsed: {time_elapsed} of {estimated_effort} hours

Provide feedback on the programmer's progress. Is the task on track?
Are there any concerns or suggestions for improvement?
""",
            "learning_recommendation": """
You are a Project Manager agent recommending learning opportunities for programmers.
Based on their performance and expertise, suggest appropriate learning activities.

Programmer: {programmer_id}
Expertise: {expertise}
Skill levels: {skill_levels}
Recent performance: {performance_metrics}

What learning activities would help this programmer improve?
Provide 1-2 specific recommendations with brief explanations.
"""
        }
    
    async def _load_model(self) -> None:
        """Load the Phi-3-mini model"""
        self.logger.info(f"Loading model from {self.model_path}")
        
        try:
            # Load tokenizer and model
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            self.logger.info("Model loaded successfully")
        except Exception as e:
            self.logger.error(f"Failed to load model: {str(e)}")
            raise
    
    async def generate_response(self, prompt: str, max_length: int = 512) -> str:
        """Generate a response using the Phi-3-mini model"""
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            
            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    max_new_tokens=max_length,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True
                )
            
            # Decode and return response
            response = self.tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            return response.strip()
        except Exception as e:
            self.logger.error(f"Error generating response: {str(e)}")
            return f"Error: {str(e)}"
    
    async def generate_thought(self, thought_type: ThoughtType, content_prompt: str) -> Thought:
        """Generate an internal thought using the agent's LLM"""
        # Create a thought-specific prompt
        prompt = f"As a Project Manager, {content_prompt}\n\nThought:"
        
        # Generate content using the model
        content = await self.generate_response(prompt, max_length=256)
        
        # Create and store the thought
        thought = Thought(
            thought_type=thought_type,
            content=content,
            confidence=0.7,  # Default confidence
            metadata={"prompt": content_prompt}
        )
        
        # Add to memory
        self.memory.add_thought(thought)
        
        return thought
    
    async def create_project(self, name: str, description: str, end_date: Optional[datetime] = None) -> Project:
        """Create a new project"""
        project = Project(
            name=name,
            description=description,
            end_date=end_date
        )
        
        self.projects[project.project_id] = project
        self.current_project_id = project.project_id
        
        # Log the creation
        self.logger.info(f"Created project: {name} ({project.project_id})")
        
        # Generate a thought about the project
        await self.generate_thought(
            ThoughtType.PLAN,
            f"I need to plan the project '{name}'. {description}"
        )
        
        return project
    
    async def create_task(self, project_id: str, title: str, description: str, 
                        priority: TaskPriority, required_expertise: List[str],
                        estimated_effort: float = 1.0) -> Task:
        """Create a new task in a project"""
        if project_id not in self.projects:
            raise ValueError(f"Project {project_id} not found")
        
        task = Task(
            title=title,
            description=description,
            priority=priority,
            required_expertise=required_expertise,
            estimated_effort=estimated_effort
        )
        
        # Add to project
        self.projects[project_id].tasks[task.task_id] = task
        
        # Log the creation
        self.logger.info(f"Created task: {title} ({task.task_id}) in project {project_id}")
        
        return task
    
    async def register_programmer(self, agent_id: str, expertise: List[str]) -> None:
        """Register a programmer with the project manager"""
        profile = ProgrammerProfile(
            agent_id=agent_id,
            expertise=expertise,
            skill_levels={skill: 0.5 for skill in expertise}  # Default skill level
        )
        
        self.programmer_profiles[agent_id] = profile
        
        # Add to current project if exists
        if self.current_project_id:
            self.projects[self.current_project_id].team_members.append(agent_id)
        
        self.logger.info(f"Registered programmer: {agent_id} with expertise: {expertise}")
    
    async def register_specialist(self, agent_id: str, specialist_type: str) -> None:
        """Register a specialist with the project manager"""
        self.specialists[specialist_type] = agent_id
        self.logger.info(f"Registered specialist: {agent_id} of type: {specialist_type}")
    
    async def assign_task(self, task_id: str, project_id: Optional[str] = None) -> None:
        """Assign a task to the most suitable programmer"""
        # Use current project if not specified
        if project_id is None:
            if self.current_project_id is None:
                raise ValueError("No current project")
            project_id = self.current_project_id
        
        if project_id not in self.projects:
            raise ValueError(f"Project {project_id} not found")
        
        project = self.projects[project_id]
        
        if task_id not in project.tasks:
            raise ValueError(f"Task {task_id} not found in project {project_id}")
        
        task = project.tasks[task_id]
        
        # Prepare programmer profiles for LLM
        programmer_profiles_text = ""
        for agent_id, profile in self.programmer_profiles.items():
            current_load = len(profile.current_tasks)
            expertise_match = sum(1 for skill in task.required_expertise if skill in profile.expertise)
            programmer_profiles_text += f"- {agent_id}: Expertise: {profile.expertise}, Current tasks: {current_load}, Expertise match: {expertise_match}/{ len(task.required_expertise)}\n"
        
        # Create prompt for task assignment
        prompt = self.prompt_templates["task_assignment"].format(
            project_name=project.name,
            task_title=task.title,
            task_description=task.description,
            required_expertise=", ".join(task.required_expertise),
            programmer_profiles=programmer_profiles_text
        )
        
        # Generate assignment decision
        assignment_decision = await self.generate_response(prompt)
        
        # Extract assigned programmer (simple extraction - in production use more robust parsing)
        # This is a simplified example - in practice, use structured output format
        assigned_to = None
        for agent_id in self.programmer_profiles.keys():
            if agent_id in assignment_decision:
                assigned_to = agent_id
                break
        
        # If no clear assignment, use fallback logic
        if not assigned_to and self.programmer_profiles:
            # Fallback: find least loaded programmer with most matching skills
            best_score = -1
            for agent_id, profile in self.programmer_profiles.items():
                expertise_match = sum(1 for skill in task.required_expertise if skill in profile.expertise)
                workload_factor = 1.0 / (len(profile.current_tasks) + 1)
                score = expertise_match * workload_factor
                
                if score > best_score:
                    best_score = score
                    assigned_to = agent_id
        
        if not assigned_to:
            self.logger.warning(f"Could not assign task {task_id} - no suitable programmer found")
            return
        
        # Update task assignment
        task.assigned_to = assigned_to
        task.status = TaskStatus.ASSIGNED
        
        # Update programmer profile
        self.programmer_profiles[assigned_to].current_tasks.append(task_id)
        
        # Generate thought about the assignment
        await self.generate_thought(
            ThoughtType.DECISION,
            f"I assigned task '{task.title}' to {assigned_to} because {assignment_decision}"
        )
        
        # Create task assignment context
        task_context = Context(
            context_type=ContextType.TASK,
            source_agent=self.agent_id,
            target_agents=[assigned_to],
            content={
                "action": "task_assignment",
                "task_id": task_id,
                "project_id": project_id,
                "task": task.dict()
            }
        )
        
        # Send context (requires message broker to be passed)
        # await self.send_context(task_context, message_broker)
        
        self.logger.info(f"Assigned task {task_id} to {assigned_to}")
    
    async def evaluate_progress(self, task_id: str, progress_report: str, project_id: Optional[str] = None) -> str:
        """Evaluate a programmer's progress on a task"""
        # Use current project if not specified
        if project_id is None:
            if self.current_project_id is None:
                raise ValueError("No current project")
            project_id = self.current_project_id
        
        if project_id not in self.projects:
            raise ValueError(f"Project {project_id} not found")
        
        project = self.projects[project_id]
        
        if task_id not in project.tasks:
            raise ValueError(f"Task {task_id} not found in project {project_id}")
        
        task = project.tasks[task_id]
        
        if not task.assigned_to:
            raise ValueError(f"Task {task_id} is not assigned to anyone")
        
        # Calculate time elapsed
        time_elapsed = (datetime.now() - task.created_at).total_seconds() / 3600  # in hours
        
        # Create prompt for progress evaluation
        prompt = self.prompt_templates["progress_evaluation"].format(
            programmer_id=task.assigned_to,
            task_title=task.title,
            task_description=task.description,
            progress_report=progress_report,
            task_status=task.status.name,
            time_elapsed=f"{time_elapsed:.1f}",
            estimated_effort=task.estimated_effort
        )
        
        # Generate evaluation
        evaluation = await self.generate_response(prompt)
        
        # Generate thought about the evaluation
        await self.generate_thought(
            ThoughtType.ANALYSIS,
            f"Evaluating progress on task '{task.title}': {evaluation}"
        )
        
        return evaluation
    
    async def recommend_learning(self, programmer_id: str) -> str:
        """Recommend learning opportunities for a programmer"""
        if programmer_id not in self.programmer_profiles:
            raise ValueError(f"Programmer {programmer_id} not found")
        
        profile = self.programmer_profiles[programmer_id]
        
        # Create prompt for learning recommendation
        prompt = self.prompt_templates["learning_recommendation"].format(
            programmer_id=programmer_id,
            expertise=", ".join(profile.expertise),
            skill_levels=json.dumps(profile.skill_levels),
            performance_metrics=json.dumps(profile.performance_metrics)
        )
        
        # Generate recommendation
        recommendation = await self.generate_response(prompt)
        
        # Generate thought about the recommendation
        await self.generate_thought(
            ThoughtType.LEARNING,
            f"Learning recommendation for {programmer_id}: {recommendation}"
        )
        
        return recommendation
    
    async def _process_message(self, context: Context) -> None:
        """Process a message context"""
        if "action" not in context.content:
            self.logger.warning(f"Message context missing 'action': {context.context_id}")
            return
        
        action = context.content["action"]
        
        if action == "task_update":
            # Process task update from programmer
            task_id = context.content.get("task_id")
            project_id = context.content.get("project_id", self.current_project_id)
            
            if not task_id or not project_id or project_id not in self.projects:
                return
            
            project = self.projects[project_id]
            
            if task_id not in project.tasks:
                return
            
            task = project.tasks[task_id]
            
            # Update task status if provided
            if "status" in context.content:
                try:
                    new_status = TaskStatus[context.content["status"]]
                    task.status = new_status
                except (KeyError, ValueError):
                    pass